In [1]:
import pandas as pd
import numpy as np
import feather

In [2]:
dist_col_names = ['seq1','seq2','dist']
dist_col_types = {'seq1':'object',
                  'seq2':'object',
                  'dist':'float32'}
dist_df = pd.read_csv('data/processed/aligned_combined.dist',
                      delim_whitespace=True,
                      header=None, 
                      names=dist_col_names,
                      dtype=dist_col_types)
print(dist_df.info())

dist_df['seq1'] = dist_df['seq1'].astype('category')
dist_df['seq2'] = dist_df['seq2'].astype('category')

print(dist_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14723451 entries, 0 to 14723450
Data columns (total 3 columns):
seq1    object
seq2    object
dist    float32
dtypes: float32(1), object(2)
memory usage: 280.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14723451 entries, 0 to 14723450
Data columns (total 3 columns):
seq1    category
seq2    category
dist    float32
dtypes: category(2), float32(1)
memory usage: 112.4 MB
None


In [3]:
before_specimens = pd.read_csv("data/original/before_schindel2011.tsv",
                              sep="\t",
                              usecols=['accession','scientific_name','trace_count'])
before_specimens['dataset'] = 'before_schindel2011'
                                        
usnm_specimens = pd.read_csv("data/original/schindel2017.tsv",
                              sep="\t",
                              usecols=['accession','scientific_name', 'trace_count'])
usnm_specimens['dataset'] = 'schindel2017'

specimen_key = before_specimens.append(usnm_specimens)
specimen_key = specimen_key[specimen_key['trace_count'] > 1][['accession','scientific_name','dataset']]

print(len(specimen_key))
print(specimen_key.head())

5427
   accession      scientific_name              dataset
13  JN419247  Haplospiza unicolor  before_schindel2011
14  JN419246  Haplospiza unicolor  before_schindel2011
15  JN419245   Diglossa sittoides  before_schindel2011
16  JN419244   Diglossopis cyanea  before_schindel2011
17  JN419243   Diglossopis cyanea  before_schindel2011


In [4]:
new_dist_df = pd.merge(dist_df, specimen_key, left_on='seq1',right_on='accession')
new_dist_df.drop('accession', axis=1, inplace=True)
del dist_df

combined_dist_df = pd.merge(new_dist_df, specimen_key, left_on='seq2',right_on='accession', suffixes=('1','2'))
combined_dist_df.drop('accession', axis=1, inplace=True)
del new_dist_df
print(combined_dist_df.head())
print(len(combined_dist_df))

       seq1      seq2      dist     scientific_name1             dataset1  \
0  JN419246  JN419247  0.002882  Haplospiza unicolor  before_schindel2011   
1  JN419245  JN419247  0.100900   Diglossa sittoides  before_schindel2011   
2  JN419244  JN419247  0.079250   Diglossopis cyanea  before_schindel2011   
3  JN419243  JN419247  0.079250   Diglossopis cyanea  before_schindel2011   
4  GU571987  JN419247  0.143500    Muscicapa striata  before_schindel2011   

      scientific_name2             dataset2  
0  Haplospiza unicolor  before_schindel2011  
1  Haplospiza unicolor  before_schindel2011  
2  Haplospiza unicolor  before_schindel2011  
3  Haplospiza unicolor  before_schindel2011  
4  Haplospiza unicolor  before_schindel2011  
14723451


In [5]:
print(combined_dist_df.info())
category_fields = ['seq1','seq2',
                   'scientific_name1','scientific_name2',
                   'dataset1','dataset2']

for field in category_fields:
    combined_dist_df[field] = combined_dist_df[field].astype("category")

print(combined_dist_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14723451 entries, 0 to 14723450
Data columns (total 7 columns):
seq1                object
seq2                object
dist                float32
scientific_name1    object
dataset1            object
scientific_name2    object
dataset2            object
dtypes: float32(1), object(6)
memory usage: 842.5+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 14723451 entries, 0 to 14723450
Data columns (total 7 columns):
seq1                category
seq2                category
dist                float32
scientific_name1    category
dataset1            category
scientific_name2    category
dataset2            category
dtypes: category(6), float32(1)
memory usage: 309.0 MB
None


In [6]:
import feather
path = 'data/processed/distance_df.feather'

In [7]:
feather.write_dataframe(combined_dist_df, path)

In [8]:
combined_dist_df = feather.read_dataframe(path)
print(combined_dist_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14723451 entries, 0 to 14723450
Data columns (total 7 columns):
seq1                category
seq2                category
dist                float32
scientific_name1    category
dataset1            category
scientific_name2    category
dataset2            category
dtypes: category(6), float32(1)
memory usage: 196.7 MB
None


In [9]:
before_dist_df = combined_dist_df[(combined_dist_df['dataset1'] == 'before_schindel2011') & \
                                  (combined_dist_df['dataset2'] == 'before_schindel2011')]
usnm_dist_df = combined_dist_df[(combined_dist_df['dataset1'] == 'schindel2017') & \
                                (combined_dist_df['dataset2'] == 'schindel2017')]

In [10]:
before_orgs = pd.DataFrame(before_specimens.groupby('scientific_name').size())
before_orgs.columns = ['count']
print(before_orgs.head())

usnm_orgs = pd.DataFrame(usnm_specimens.groupby('scientific_name').size())
usnm_orgs.columns = ['count']

combined_orgs = pd.DataFrame(specimen_key.groupby('scientific_name').size())
combined_orgs.columns = ['count']

dataset_dict = {'before_schindel2011':
                 {'org_df': before_orgs,
                 'dist_df': before_dist_df},
                'schindel2017':
                 {'org_df': usnm_orgs,
                 'dist_df': usnm_dist_df},
                'combined':
                 {'org_df': combined_orgs,
                 'dist_df': combined_dist_df}}

                     count
scientific_name           
Abrornis humei           4
Abrornis inornata        6
Abrornis proregulus     10
Acanthis flammea        17
Acanthis hornemanni     11


In [11]:
import datetime as dt

In [12]:
def max_intra(organism, count, df):
    if count > 1:
        return df[(df.scientific_name1 == organism) & 
               (df.scientific_name2 == organism)]['dist'].max()
    else:
        return np.nan
    
def min_inter(organism, df):
    return df[((df.scientific_name1 == organism) | 
               (df.scientific_name2 == organism)) & 
               (df.scientific_name1 != df.scientific_name2)]['dist'].min()

In [13]:
for dataset in dataset_dict:
    print(dataset)
    start_time = dt.datetime.now()
    
    org_df = dataset_dict[dataset]['org_df']
    distance_df = dataset_dict[dataset]['dist_df']

    org_df['max_intra'] = org_df.apply(lambda x: max_intra(x.name, x['count'], distance_df), axis=1)
    org_df['min_inter'] = org_df.apply(lambda x: min_inter(x.name, distance_df), axis=1)
    
    org_df.reset_index(level=0, inplace=True)
    distance_filename = "data/processed/"+ dataset + "_distance_summary.tsv"
    org_df.to_csv(distance_filename, sep='\t', index=False)    

    end_time = dt.datetime.now()
    print('Elapsed time for analysis:',(end_time - start_time))

combined
Elapsed time for analysis: 0:05:20.924965
before_schindel2011
Elapsed time for analysis: 0:01:07.142188
schindel2017
Elapsed time for analysis: 0:01:04.674279
